In [ ]:
import os
import warnings
import glob
import numpy as np
import pandas as pd
from skimage import io
import gaussianfunctions_KM as gf
from skimage.measure import label, regionprops,regionprops_table
from skimage import data, util, measure
from scipy import ndimage
import numpy.ma as ma
from matplotlib.patches import Ellipse
from skimage.draw import ellipse
from skimage import draw
from scipy.ndimage import gaussian_filter
warnings.filterwarnings("ignore")
io.use_plugin('matplotlib')

def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

def get_subdirs(dir):
    return next(os.walk(dir))[1]


def gaussian(x, amplitude, mean, stddev, offset):
    return (amplitude * np.exp(-((x - mean) / 4 / stddev)**2)+offset)    


       

directory = "path"
conditions = get_subdirs(directory)

fov_list = []
track_id_list = []
for condition in conditions:

    os.chdir(directory +"\\"+condition)

    #get image name
    image_name = ''.join([f for f in glob.glob("*ome.tif")])
    
    #get trackmate result file name
    result_file_name = glob.glob("*.csv")
    
    
    if len(result_file_name)>0:
        for s in result_file_name:
            #read in movie
            movie = io.imread(image_name).astype(np.float64)
            
            #get YAP channel
            YAP_channel = movie[:,:,:,2]  

            #read in trackmate result file
            trackmate_result = pd.read_csv(s)

            #remove the first two rows that are duplicates of the header (default trackmate output format)
            tm = trackmate_result.drop([0, 1,2])

            #change frames to float
            tm["FRAME"]= tm["FRAME"].astype(float)
            
            #make a copy of the trackmate df to add spot quantifications
            tm_copy = tm
            
            #add new columns to append results
            tm_copy["new_frame"]="" 
            tm_copy['YAP_integral_int']=""
            tm_copy['YAP_dilute_phase_mean']=""

            #extract all IDs
            track_list = tm["ID"].to_list()
            
            #convert all IDs to integers 
            spot_list = [float(i) for i in track_list]

            #iterate through every track
            for i in spot_list:
                
                #convert track IDs to floats              
                tm["ID"]= tm["ID"].astype(float)
            
                #select track
                current = tm.loc[tm["ID"] == i]

                #extract all frames of the track
                frame_list = current['FRAME'].to_list()  
                int_frame_list = [int(i) for i in frame_list]

                #iterate through frames
                for d in int_frame_list:
                                   
                                    f = int(d)
                                    
                                    #get x,y coordinate of the spot
                                    current_df = current[current['FRAME']==float(f)]
                                    x_pos = current_df["POSITION_Y"].values[0]
                                    y_pos = current_df["POSITION_X"].values[0]
                                    
                                    #get the current frame of the movie
                                    current_img = YAP_channel[f,:,:]
                                    
                                    #select a small region around the spot on the image
                                    y_min = round(float(y_pos)) - 7
                                    y_max = round(float(y_pos)) + 8
                                    x_min = round(float(x_pos)) - 7
                                    x_max = round(float(x_pos)) + 8

                                    local_img = current_img[x_min:x_max, y_min:y_max]

                                    #-----------Extimate fitting parameters and fit guassian to trackmate-identified YAP spots-------------
                                    #get local background from a slightly larger area 
                                    y_min_global = np.int64(float(y_pos) - 10)
                                    y_max_global = np.int64(float(y_pos) + 11)
                                    x_min_global = np.int64(float(x_pos) - 10)
                                    x_max_global = np.int64(float(x_pos) + 11)
                                    local_img_global = current_img[x_min_global:x_max_global, y_min_global:y_max_global]

                                    thresh = np.median(local_img_global)*0.5

                                    #For spots that are at the edge of a nucleus, the gaussian fit open recognizes the nucleus edge instead of the spot.
                                    #To avoid this, fill in pixel outside of the nucleus with the average nuclear background value 
                                    local_img_masked = ma.masked_where(local_img <= thresh, local_img)
                                    local_img_masked_filled = local_img_masked.filled(fill_value=np.mean(local_img_masked))

                                    #estimate offset
                                    off_guess_img = np.median(local_img_global[local_img_global>100]) 
                                    local_median_img = off_guess_img

                                    #compute min gaussheight to be at least 10% above offset
                                    height_low_bound_img = (1.1 * off_guess_img) - off_guess_img; 
                                    
                                    #get local maximum to estimate amplitude
                                    local_max_value_img = np.max(local_img_masked)
                                    amp_guess_img = local_max_value_img - off_guess_img                           

                                    # inital guesses for gaussian fit
                                    initial_guess_img = (amp_guess_img, 7, 7, 1.0, 1.0, off_guess_img)

                                    #set boundary conditions for gaussian fitting
                                    bound_tup_img = ([height_low_bound_img, 7, 7, 1, 1, 0], [+np.Inf, 8, 8, 2.5, 2.5, +np.Inf])

                                    #Generate grid for gaussian fitting
                                    x_local_gauss_img = np.linspace(0, 15 - 1, 15)
                                    y_local_gauss_img = np.linspace(0, 15 - 1, 15)
                                    x_local_gauss_img,y_local_gauss_img = np.meshgrid(x_local_gauss_img, y_local_gauss_img)

                                    #Fit 2D gaussain to local YAP image
                                    param_img = gf.fit(gf.func, (x_local_gauss_img,y_local_gauss_img), local_img_masked_filled, initial_guess_img, bound_tup_img) ##Fit gaussian


                    
                                    #-------------quantify YAP hub integral intensity and dilute phase intensity----------------
                                    #subtract background from YAP image
                                    local_img_global_bkgd_subtr = local_img - param_img.offset

                                    #if background subtraction results in negative values, replace them with zeros
                                    local_img_global_bkgd_subtr_masked = ma.masked_where(local_img_global_bkgd_subtr <= 0, local_img_global_bkgd_subtr)
                                    local_img_global_bkgd_subtr_masked_filled = local_img_global_bkgd_subtr_masked.filled(fill_value=0)

                                    # generate an ellipse array based on gaussian fit parameters (will be used as image mask)
                                    ellipse_outline = Ellipse((param_img.center_x, param_img.center_y), width=param_img.sigma_x*3, height=param_img.sigma_y*3,linewidth=1, edgecolor='r', facecolor='none')
                                    ellipse_array = np.zeros((local_img_global_bkgd_subtr_masked_filled.shape[0], local_img_global_bkgd_subtr_masked_filled.shape[1]))
                                    rr, cc = ellipse(param_img.center_y, param_img.center_x, param_img.sigma_y*3/2, param_img.sigma_x*3/2)
                                    ellipse_array[rr,cc]=1
                                    ellipse_label = label(ellipse_array)
                                    
                                    #mask local YAP image outside of gaussian fit
                                    local_img_global_bkgd_subtr_masked_filled_spot = ma.masked_where(ellipse_label == 0, local_img_global_bkgd_subtr_masked_filled)
                                    
                                    #fill masked values with zeros and quantify sum intensity of YAP hub pixel (=dense phase YAP intensity)
                                    local_img_global_bkgd_subtr_masked_filled_spot_quant = local_img_global_bkgd_subtr_masked_filled_spot.filled(fill_value=0)
                                    dense_phase_intensity = np.sum(local_img_global_bkgd_subtr_masked_filled_spot_quant)
                
                                    #Quantify dilute phase intensity in vicinity to the YAP hub
                                    local_img_global_bkgd_subtr_masked_filled_dilute_quant = ma.masked_where(ellipse_label == 1, local_img_global_bkgd_subtr_masked_filled)
                                    dilute_phase_intensity = np.mean(local_img_global_bkgd_subtr_masked_filled_dilute_quant) 

                                    #------------enter results into dataframe-----------
                                    #select row in dataframe
                                    matched_row = (tm_copy['FRAME'] == float(f))&(tm_copy['ID']==float(i))

                                    #enter quantifications into existing dataframe
                                    tm_copy.loc[matched_row,'new_frame'] = str(f)

                                    #if image contained no YAP spots, enter zeros, else enter results
                                    if param_img.amp == 0:
                                        tm_copy.loc[matched_row,'YAP_integral_int'] = str([0])
                                        tm_copy.loc[matched_row,'YAP_dilute_phase_mean'] = str([0])
                                        
                                    else:
                                        tm_copy.loc[matched_row,'YAP_integral_int'] = str(dense_phase_intensity)
                                        tm_copy.loc[matched_row,'YAP_dilute_phase_mean'] = str(dilute_phase_intensity)


            tm_copy.to_csv(directory + "\\"+s+"_result.csv")  
  